# Setup START
***
Skipped in slideshow mode, run manually before!

Ideally, the code would be placed with their corresponding content slides.\
Unfortunately, RISE slideshow mode will not execute SKIP slides and there is no other way to hide them.

In [ ]:
%pip install --upgrade pip
# OpenAI Gym related:
%pip install cmake
%pip install atari_py 
'''OpenAI devs actually messed up an indentation
    in the video recorder class in 0.18.x.
    If you want to see videos, wait for a patch or
    use 0.17.x.
'''
%pip install gym[box2d]==0.17.3
%pip install pyvirtualdisplay
%pip install tqdm

# Gridworld related:
%pip install pymdptoolbox

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !apt-get install -y xvfb python-opengl > /dev/null 2>&1

### Setup: OpenAI Gym

In [ ]:
from IPython import display
import numpy as np
import base64
import io
import gym
from tqdm import tqdm
from gym.wrappers import Monitor

if IN_COLAB:
    from pyvirtualdisplay import Display
    d = Display()
    d.start()

'''
Below is a solution to record and display videos for
OpenAI Gym environments.
Why the hassle?
    It works both in Jupyter Notebook and Google Colab!
(The author would like to note how painful it has been
developing this)
'''    

def display_video_from_monitor(monitor):
    '''
    Converts all videos in a monitor to HTML videos
    '''
    if len(monitor.videos) == 0:
        print("No videos to render!")
        return
    for f in monitor.videos:
        video = io.open(f[0], 'r+b').read()
        encoded = base64.b64encode(video)

        display.display(display.HTML(data="""
            <video alt="test" controls>
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>
            """.format(encoded.decode('ascii'))))
    return

def make_video(env, model, max_steps=None):
    mon = Monitor(env, "/data/videos/", force=True)
    state = mon.reset()
    i = 0
    while True:
        action = model(state)
        state, r, d, _ = mon.step(action)
        if d: break
        i+=1
        if max_steps is not None and i>=max_steps: break
    if mon.stats_recorder.done == False:    
        mon.stats_recorder.save_complete()
        mon.stats_recorder.done = True
    mon.reset()
    display_video_from_monitor(mon)
    return mon

### Setup: Grid World

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches
from mdptoolbox import mdp

class Gridworld():
    def __init__(self, actions, w=4, h=3, rho=1.0, step_reward=0.0, cell_rewards={},
               terminal_states=[], blocked_cells=[]):
        self.actions = actions
        self.w, self.h = w, h
        self.rho = rho
        self.step_reward = step_reward
        self.cell_rewards = cell_rewards
        self.terminal_states = {tuple(x) for x in terminal_states}
        self.blocked_cells = {tuple(x) for x in blocked_cells}

        self.grid = np.zeros((h,w))
        self.states = np.array([[i,j] for i in range(h) for j in range(w)])
        self.num_states = w*h
        self.num_actions = len(actions)
        self.p = np.zeros((self.num_actions, self.num_states, self.num_states), "float32")
        self.r = np.zeros((self.num_actions, self.num_states, self.num_states), "float32")
        self._fill_p()
        self._fill_r()

    def _fill_p(self):
        # computing transition matrix:
        for a in range(self.num_actions):
            for i in range(self.num_states):
                # simulate performing action a_ in state i
                # action a is performed with prob. rho,
                # with prob. 1-rho, any other action is chosen,
                # which results in prob. (1-rho)/(num_moves-1) for
                # a specific action a_ != a
                for a_ in range(self.num_actions):
                    rh, rw = self.simulate_move(self.actions[a_], self.states[i])
                    j_ = rh*self.w + rw
                    prob = self.rho if a == a_ else (1-self.rho)/(self.num_actions-1)
                    self.p[a, i, j_] += prob
                # due to numerical inaccuracies, we need to ensure that each row
                # adds up to exactly 1, not 0.999995, because the solvers
                # really dont seem to like that
                row_sum = self.p[a,i].sum()
                if row_sum != 1:
                    if not np.allclose(row_sum,1, rtol=1e-5, atol=1e-5):
                        raise ValueError(f"Probability error at action{a}, state {i}"
                                         f" with prob. {row_sum}")
                    self.p[a,i] /= row_sum
        return
        # make sure we can't ever leave terminal states
        for (i,j) in self.terminal_states:
            # transition probability from state (i,j) to any other state 
            # must be zero, but one for (i,j)
            index = i*self.w+j
            self.p[:,index,:] = 0
            self.p[:, index, index] = 1

    def _fill_r(self):
        # transitions always give step_reward
        self.r.fill(self.step_reward)
        
        # ending on a rewarded cell gives the corresponding reward
        for (i,j), reward in self.cell_rewards.items():
            self.r[:,:, i*self.w+j] += reward

        # terminal cells do not give ANY reward after having reached them
        for (i,j) in self.terminal_states:
            k = i*self.w+j
            self.r[:, k, :] = 0

        # should staying on cells still give the associated rewards?
        for i in range(self.num_actions):
            np.fill_diagonal(self.r[i],0)

    def simulate_move(self, move, state):
        if self._is_blocked(state): return state
        if self._is_terminal(state): 
            return state
        nm = np.clip(np.array(move)+np.array(state), 0, (self.h-1, self.w-1))
        if self._is_blocked(nm):
            return state
        return nm

    def solve(self, discount=0.99, verbose=False):
        '''
        Solves the underlying MDP using Policy Iteration
        Returns an object with the optimal policy. See
        `mdp.PolicyIteration` doc for details.
        '''
        vi = mdp.PolicyIteration(self.p, self.r, discount)
        if verbose:
            vi.setVerbose()
        vi.run()
        return vi
    
    def _is_blocked(self, cy, cx=None):
        # cx: position along grid width
        if cx is None:
            cy, cx = cy
        return (cy, cx) in self.blocked_cells

    def _is_terminal(self, cy, cx=None):
        if cx is None:
            cy, cx = cy
        return (cy, cx) in self.terminal_states
    
    def _plot_policy(self, policy):
        a_l = 0.6 # arrow length
        for (i,j), a in list(zip(self.states, policy)):
            if self._is_terminal(i,j) or self._is_blocked(i,j): continue
           
            dh, dw = self.actions[a]
            if dh == -1:
                y_off = 1 - (1-a_l)/2.
                x_off = 0.5
            elif dh == 0:
                y_off = 0.5
                if dw == -1:
                    x_off = 1 - (1-a_l)/2.0
                elif dw == 1:
                    x_off = (1-a_l)/2.0
                else: raise ValueError("Invalid action to plot!")
            elif dh == 1:
                y_off = (1-a_l)/2.0
                x_off = 0.5
            else: raise ValueError("Invalid action to plot")
            plt.arrow(j+x_off, i+y_off, dw*a_l, dh*a_l, width=0.1, 
                      head_width=0.4, head_length=0.6*0.35, fc="k", ec="k",
                      length_includes_head=True, linewidth=0, overhang=0.3, 
                      zorder=10)
        return

    def _plot_terminal_states(self):
        for (i,j) in self.terminal_states:
            c = plt.Circle((j+0.5,i+0.5), radius=0.3, fill=False, ec="black",
                          lw=3, zorder=6)
            plt.gca().add_patch(c)

    def _plot_blocked_cells(self):
        for (i,j) in self.blocked_cells:
            rect = patches.Rectangle((j,i),1,1, zorder=1, color=(1,219/255,147/255))
            plt.gca().add_patch(rect)        
            
    def _plot_rewards(self, resize_factor):
        if len(self.cell_rewards) == 0: return

        neg_cm = plt.get_cmap("Reds")
        pos_cm = plt.get_cmap("Greens")
        rmax = max(abs(x) for x in self.cell_rewards.values())
        rmin = -rmax
        for (i,j), r in self.cell_rewards.items():
            plt.text(j+0.05,i+0.95, f"{r:.1f}", zorder=10, weight="bold",
                    size=8*resize_factor)
            # to get the right color, we normalize to 0,255
            r_norm = 1.0 * (r-rmin)/(rmax-rmin)
            cm = pos_cm
            if r < 0:
                r_norm = 1.0 - r_norm
                cm = neg_cm
            col = cm(r_norm)
            rect = patches.Rectangle((j,i),1,1, zorder=1, color=col)
            plt.gca().add_patch(rect)   
            
    def plot(self, policy=None, height=None, width=None):
        if height is None and width is None:
            height = self.h
            width = self.w
        if height is None:
            height = self.h*(width/self.w)
        if width is None:
            width = self.w*(height/self.h)
        rf = height/self.h
        fig, ax = plt.subplots(figsize=(width, height))
        if policy is not None:
            self._plot_policy(policy)
        self._plot_terminal_states()
        self._plot_blocked_cells()
        self._plot_rewards(rf)
        ax.grid(zorder=10, lw=1)
        

        plt.xlim(0, self.w)
        plt.ylim(0, self.h)
        plt.tick_params(axis='both', which='both', bottom=False, top=False, 
                        labelbottom=False, right=False, left=False, labelleft=False)
        plt.xticks(range(self.w))
        plt.yticks(range(self.h))
        ax.invert_yaxis()
        plt.title("Grid world")
        plt.show()

# Setup END
***
(Click this cell, and do "Run all above")

# An introduction to Reinforcement Learning
***
Based on:
* **MIT 6.S091**: Introduction to Deep Reinforcement Learning, by Lex Fridman
* **UC Berkeley CS 287**: Advanced Robotics, by Pieter Abbeel

In [ ]:
try:
    IN_COLAB
    print("Setup was run :)")
except:
    print("Don't forget to run the setup!")

# Agenda
***
1. What is Deep RL?
    * Deep Learning vs Deep Reinforcement Learning
2. Types of learning
    * Supervised vs unsupervised vs RL
    * RL in Humans, why are we so good?
3. Reinforcement Learning Framework
    * Agent, Action, State, Reward
4. Environments & Actions
    * Observability, Single VS Multi Agent, Deterministic/Stochastic, Static/Dynamic, Discrete/Continuous
5. RL in the real world
    * Two open problems:
        * Make better simulations
        * Make algorithms that generalize better from simulation to real world
6. Components of an RL agent
    * Policy, the behavior function, probability of taking $a_t$ when in state $s_t$
    * Value function, how good is each state/action in terms of expected future reward
        * $V_\pi(s)=E\left[\sum_{t=0}^{\infty}\gamma^t r_t\mid s_0 = s\right]$
    * Model, agent's representation of the environment
7. Meaning of Life: Maximize Reward
    * Future reward, discounted reward
        * Sparse reward problem
        * Credit assignment problem
8. Robot in a room/Gridworld:
    * Example to show the impact of reward structure and environment on the optimal policy
    * WE HAVE CODE NOW, WOOOOO
9. Types of RL (several ways to taxonomize exist!)
    * Model-based
        * learn a model of the world through interaction, then plan ahead using the model
    * Value-based (off policy)
        * Learn the values for states or states/actions
            * Constantly update how good it is to take action $a$ in state $s$
            * Act by choosing the best action in state. Exploration is needed!
    * Policy-based
        * Learning the right behavior function that maps state to action
        * Act by sampling policy, exploration already included
10. Taxonomy by OpenAI
    * Is this needed?
    
TODO:
DQN oder A2C
+ Daran gebundene Challenge

# What is Deep Reinforcement Learning?
***
Last lecture we heard about **Deep Learning**:
* Learn from examples, supervised
* Find patterns, representations

**Reinforcement Learning** is about:
* Solving sequential decision making problems
    * Comprehend the world and *act* based on that
* How? Through trial and error in a world that provides rewards

**Deep RL** is RL + Neural networks

# Types of learning
***
* Supervised, Semi-supervised
* Unsupervised
* Reinforcement learning

In each case of machine learning, the loss function **supervises** the learning
* Someone has to say what's good and what's bad!

It's more about the amount of human effort involved in training.
* Supervised learning: Teach by example
    * Learning patterns from data
* Reinforcement learning: Teach by experience
    * Learning from a world by exploration

# Reinforcement learning in humans
***
Humans appear to learn from very few examples of trial and error.
* How, is still an open question

There are several possible answers to why humans might learn so fast
* Data & Hardware: 230 million years of trial and error
* Imitation learning: Observation of other humans
* Algorithms: Better than backpropagation and gradient descent

# Reinforcement learning framework
***
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Markov_diagram_v2.svg/640px-Markov_diagram_v2.svg.png" style="float: right;">

At each step, the **agent**:
* Executes an **action**
* Observes a new **state**
* Receives a **reward**

# Environments and actions
***
* Fully Observable (Chess) vs Partially Observable (Poker)
* Single Agent (Atari) vs Multi Agent (Deep Traffic)
* Deterministic (CartPole) vs Stochastic (Deep Traffic)
* Static (Chess) vs Dynamic (Deep Traffic)
* Discrete (Chess) vs Continuous (CartPole)

# The challenge for real world RL applications
***
The world an agent might learn from is a model designed by us
* Agents live in a (partial) simulation of our world

To apply RL agents to the real world, they need to *transfer* their experience
* Two options:
    * Improve the simulations
    * Improve algorithms to better generalize from simulation to real world

# Components of an RL agent
***
**Policy $\pi$**, an agent's behavior function:\
$\pi(s,a)=P\left(a_t=a\mid s_t=s\right)$

**Value function $Q$**: How good is each state and/or action\
$\displaystyle Q^\pi(s,a)=E\left[R\mid s,a,\pi\right]$

**Reward $R_a(s,s')$**:\
The immediate reward after transition from $s$ to $s'$ with action $a$

**Model**: The agent's representation of the environment
<br>
<br>
<center>$s_0,a_0,r_1,s_1,a_1,r_2,\dots,s_{n-1},a_{n-1},r_n$</center>


# Meaning of life: Maximize Reward
***
The purpose of an RL agent is to maximize the future reward\
$R_t=r_t+r_{t+1}+\cdots+r_n$

Rewards are discounted by factor $\gamma$:\
$R_t=r_t+\gamma r_{t+1}+\gamma^2 r_{t+2}+\cdots+\gamma^{n-t}r_n$

Why discounted?
* A math trick to prove certain convergence properties
* Also accounts for increasing uncertainty of far out rewards

A good strategy would be to always choose the action that gives the highest future reward

# Grid World or *Robot in a room*
***
Imagine a robot in a room. It may move and find:
* Positive and negative rewards, as shown by value and color
* Terminal states, indicated by a circle
* Walls/Blocks, appearing yellow(ish)

In [ ]:
rewards = {(0,3):1,(1,3):-1}
terminal_states = rewards.keys()
blocks = [(1,1)]
grid_world = Gridworld([],w=4,h=3, cell_rewards=rewards, 
                       terminal_states=terminal_states,
                       blocked_cells=blocks)
grid_world.plot(width=8)

# Deterministic movement
***
We'll allow our robot to move UP, DOWN, LEFT and RIGHT, but each move costs
* Denoted by `step_reward=-0.04`

In [ ]:
actions = np.array([
    [-1,0], # UP
    [1,0], # DOWN 
    [0,-1], # Left
    [0,1], # Right
])

rewards = {(0,3):1,
           (1,3):-1}
terminal_states = list(rewards.keys())
blocks = [(1,1)]

grid_world = Gridworld(actions,w=4,h=3, step_reward=-0.04,
                       cell_rewards=rewards, 
                       terminal_states=terminal_states,
                       blocked_cells=blocks)

grid_world.plot(width=8)

# Markov decision process (MDP)
***
What would be the optimal move on each cell to achieve the highest possible reward?
* The theory behind Markov decision processes allows us to solve this!
    * This is where the discount factor $\gamma$ is needed

In [ ]:
result = grid_world.solve(discount=0.99)
grid_world.plot(policy=result.policy, width=8)

**Policy**: Shortest path to $+1$ reward

# Optimal policy for positive `step_reward`
***
Everything else being equal, how would a positive `step_reward` influence the agent's optimal behavior?

In [ ]:
grid_world = Gridworld(actions,w=4,h=3, step_reward=0.04,
                       cell_rewards=rewards, 
                       terminal_states=terminal_states,
                       blocked_cells=blocks)
result = grid_world.solve(discount=0.99)
grid_world.plot(policy=result.policy, width=8)

**Policy**: Never terminate!

# Adding uncertainty
***
So far, each action chosen by the agent resulted in exactly the anticipated outcome
* It's a *deterministic* world our robot lives in

However, some systems are stochastic by nature, or too complex to model deterministically
* How would uncertainty impact the optimal policy in our grid world?
* We let $\rho\in (0,1]$ denote the probability of executing the intended action $a$
    * But with prob. $1-\rho$ any other action is executed

Here is the deterministic result ($\rho=1$):

In [ ]:
grid_world = Gridworld(actions,w=4,h=3, step_reward=0.04,
                       cell_rewards=rewards, 
                       terminal_states=terminal_states,
                       blocked_cells=blocks)
result = grid_world.solve(discount=0.99)
grid_world.plot(policy=result.policy, height=5)

What happens for $\rho=0.7$ ?

In [ ]:
grid_world = Gridworld(actions,w=4,h=3, step_reward=0.04,
                       rho=0.7, cell_rewards=rewards, 
                       terminal_states=terminal_states,
                       blocked_cells=blocks)
result = grid_world.solve(discount=0.99)
grid_world.plot(policy=result.policy, height=5)

# Lessons learned from Grid World
***
1. Environment model has big impact on optimal policy
2. Reward structure has big impact on optimal policy

In [ ]:
import IPython
IPython.display.HTML('''
<iframe width="800" height="510" src="https://www.youtube.com/embed/tlOIHko8ySg" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
''')

In [ ]:
w=8; h=4; rho=1.0; step_reward=-0.04;

actions = np.array([
    [-1,0], # UP
    #[1,0], # DOWN 
    [0,-1], # Left
    [0,1], # Right
])
cell_rewards={
    (0,3): 1,
    (1,3): -100,
    #(1,6): 1,
    (2,0): -4,
}
terminal_states=[
    [0,3],
    [1,3],
]
blocked=[
    [0,5],
    [1,5],
    [2,5],
    [0,6],
    [1,6],
    [2,6],
    [0,7],
    [1,7],
    [2,7],
]

gw  = Gridworld(w=w, h=h, rho=rho, step_reward=step_reward,
                cell_rewards=cell_rewards, actions=actions,
                terminal_states=terminal_states, blocked_cells=blocked)
gw.plot(height=7)

In [ ]:
#vi = mdp.ValueIteration(gw.p, gw.r, 0.9, max_iter=1e5)
vi = mdp.PolicyIteration(gw.p, gw.r, 0.99, max_iter=50, eval_type=0)
vi.setVerbose()
vi.run()
#print(vi.policy)
gw.plot(policy=vi.policy, height=10)
#vi.V

# 3 Types of Reinforcement learning
***
**Model-based**
* Learn the model of the world, then plan on it
* Update model and re-plan often

**Value-based**
* Learn the values of states or state-action pairs
* Act by choosing best action in state
* Exploration is a needed add-on

**Policy-based**
* Learn the stochastic behavior function $\pi$
* Act by sampling the policy
* Exploration comes from the stochastic nature of the policy function

# Sample efficiency
***
<center><img src="img/slides/rl_sample_efficiency.png"/></center>

**Off-policy** are value-based agents
* Constantly update the quality of being in a certain state
* Always take the best next state
    * Except sometimes, in order to explore other states
    
**Training RL agents can be a very lengthy process**
* Training usually comes with high variance

# Actor-Critic
***
The **Actor** component takes the state of the world and returns a probability distribution over the actions
* Ideally, the best actions get high probability
* A policy-based idea

We want the **Critic** to take a state and return the estimated future rewards
* Estimates how good the choices of the actor are
* A value-based approach

They are trained together to get better at their respective tasks:
* Critic gets more accurate value estimations
* Actor assigns higher probability to high-valued actions

# Actor-Critic
***


# Placeholder
***

In [ ]:
''' 
Loosely based on:
https://www.tensorflow.org/tutorials/reinforcement_learning/actor_critic?hl=en
'''

import numpy as np
from tensorflow.keras.layers import (
    Dense, Input, Dropout    
)
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.losses import Huber
from tensorflow import keras
from tqdm import tqdm

import gym


def episode(env, model, max_steps=None):
    '''
    Runs a single episode of the model in env for at most max_steps
    Returns states, rewards, action_probabilities and the
    estimated rewards by the critic.
    '''
    states = []
    action_probs = []
    est_rewards = []
    rewards = []

    state = env.reset()
    i = 0
    max_steps = np.inf if max_steps is None else max_steps
    while i < max_steps:
        state = tf.convert_to_tensor(state)
        state = tf.expand_dims(state, 0)
        states.append(state)
        a_p, e_w = model(state)

        action = np.random.choice(env.action_space.n, p=np.squeeze(a_p))
        state, reward, done, info = env.step(action)

        action_probs.append(a_p[0, action])
        est_rewards.append(e_w[0,0])
        rewards.append(reward)

        i += 1
        if done:
            break
    env.close()
    return states, rewards, action_probs, est_rewards

def sample_action(model, state):
    '''
    Given a model and a state, samples an action from model
    '''
    state = tf.convert_to_tensor(state)
    state = tf.expand_dims(state, 0)
    a_p, e_w = model(state)
    action = np.random.choice(env.action_space.n, p=np.squeeze(a_p))
    return action

class ActorCriticDiscrete():
    '''
    Implements the Actor Critic network structure for discrete action spaces
    '''
    def __init__(self, env, model, critic_loss=keras.losses.Huber(),
                opt="Adam", val_env=None):
        self.env = env
        self.val_env = env if val_env is None else val_env
        self.model = model
        self.critic_loss = critic_loss
        self.opt = opt

    def train_episode(self, discount, max_train_steps):
        with tf.GradientTape() as tape:
            states, rewards, action_probs, est_rewards = episode(self.env, self.model, max_train_steps)
            reward = sum(rewards)

            returns = []
            discounted_sum = 0
            for r in rewards[::-1]:
                discounted_sum = r + discount * discounted_sum
                returns.insert(0, discounted_sum)
            returns = np.array(returns)
            returns = (returns - returns.mean()) / (returns.std()+0.000001)

            returns = tf.convert_to_tensor(returns, "float32")

            critic_loss = self.critic_loss(est_rewards, returns)
            al = -tf.math.log(action_probs) * (returns - est_rewards)
            actor_loss = tf.reduce_mean(al)
            loss = actor_loss + critic_loss

            gradients = tape.gradient(loss, self.model.trainable_weights)
            self.opt.apply_gradients(zip(gradients, self.model.trainable_weights))
        return reward

    def val_episode(self, val_episodes=1, max_val_steps=None):
        if max_val_steps is None:
            max_val_steps = self.val_env._max_episode_steps

        self.val_env._max_episode_steps = max_val_steps

        val_rewards = []
        for val_e in range(val_episodes):
            s, r, ap, er = episode(self.val_env, self.model, max_val_steps)
            val_rewards.append(sum(r))
        val_rewards = np.array(val_rewards)
        return val_rewards

    def sample_action(self, state):
        '''
        Convenienve wrapper to quickly run a state through model.
        Returns an action
        '''
        return sample_action(self.model, state)

    def train(self, epochs, discount, max_train_steps = 200, validate_every=5, 
              val_episodes=5, max_val_steps = 200, video=True, max_video_steps=500,
             avg_reward_goal=None):
        if max_train_steps > self.env._max_episode_steps:
            raise ValueError("The provided training environment doesn't allow"
                            f" more than {self.env._max_episode_steps} steps per"
                            " episode. You might want to change the env settings.")
        reward_history = []
        
        for epoch in tqdm(range(epochs)):

            reward = self.train_episode(discount, max_train_steps)
            if epoch % validate_every == 0:
                if video:
                    make_video(self.val_env, self.sample_action, max_video_steps)
                if val_episodes > 0:
                    vr = self.val_episode(val_episodes, max_val_steps)
                    print(f"Epoch {epoch}/{epochs}: Validation reward mean/min/max: "
                          f"{vr.mean():0.3f}, {vr.min():0.3f}, {vr.max():0.3f}")
                    if avg_reward_goal is not None and vr.mean() >= avg_reward_goal:
                        print(f"Reached goal of mean reward {avg_reward_goal}, stopping!")
                        break
        return


# OpenAI Gym
***
The `gym` package from OpenAI is a repository of multiple RL environments:
* Classic control, i.e. balancing a pole on a cart
* Robotics: Learning to pick up or stack in a 3D world
* MuJoCo: Learning movement in a physics simulator (license required)
* Atari: Huge amount of Atari 2600 games, like Breakout, SpaceInvaders or Pong
* Tons of third party environments



In [ ]:
env = gym.make('CartPole-v0')
val_env = env.unwrapped

tf.random.set_seed(1)

input_layer = Input(env.observation_space.shape)
l = Dense(128, "relu")(input_layer)
actor = Dense(env.action_space.n, "softmax", name="actor")(l)
critic = Dense(1, name="critic")(l)
model = Model(input_layer, [actor, critic])

model.summary(50)
opt = keras.optimizers.Adam(lr=0.02)
critic_loss = keras.losses.MeanSquaredError()

a2c = ActorCriticDiscrete(env, model, critic_loss, opt, 
                          val_env=val_env)

In [ ]:
a2c.train(epochs=2000, discount=0.99, 
          max_train_steps=200, validate_every=50, 
          val_episodes=5, max_val_steps=500, 
          video=True, max_video_steps=500, 
          avg_reward_goal=195)

In [ ]:
env = gym.make('LunarLander-v2')
val_env = env.unwrapped

tf.random.set_seed(1)

input_layer = Input(env.observation_space.shape)
l = Dense(128, "relu")(input_layer)
actor = Dense(env.action_space.n, "softmax", name="actor")(l)
critic = Dense(1, name="critic")(l)
model = Model(input_layer, [actor, critic])

model.summary(50)
opt = keras.optimizers.Adam(lr=0.02)
critic_loss = keras.losses.Huber()

a2c = ActorCriticDiscrete(env, model, critic_loss, opt, 
                          val_env=val_env)

In [ ]:
a2c.train(epochs=5000, discount=0.99, 
          max_train_steps=200, validate_every=50, 
          val_episodes=5, max_val_steps=200, 
          video=True, max_video_steps=500, 
          avg_reward_goal=195)